*problem: GLIBCXX 3.4.30 not found in conda environment*

solution: 
check:
strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep GLIBCXX
then change:
ln -sf /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /home/anavani/anaconda3/envs/dmcgb/bin/../lib/libstdc++.so.6

* Convert delta actions to absolute actions
```bash
python utils/robomimic_convert_action.py --dataset=robomimic/datasets/lift/ph/low_dim.hdf5
```

* Save waypoints
```bash
python utils/robomimic_save_waypoints.py --dataset=robomimic/datasets/lift/ph/low_dim.hdf5 --err_threshold=0.005
```

* Replay waypoints (save 3 videos and 3D visualizations by default)
```bash
mkdir video
python example/robomimic_waypoint_replay.py --dataset=robomimic/datasets/lift/ph/low_dim.hdf5 \
    --record_video --video_path video/lift_waypoint.mp4 --task=lift \
    --plot_3d --auto_waypoint --err_threshold=0.005

* Train DP_awe
CUDA_VISIBLE_DEVICES=0 python diffusion_policy/train.py --config-dir=config --config-name=waypoint_image_lift_ph_diffusion_policy_transformer.yaml hydra.run.dir='data/outputs/${now:%Y.%m.%d}/${now:%H.%M.%S}_${name}_${task_name}'

* Train original DP
CUDA_VISIBLE_DEVICES=1 python diffusion_policy/train.py --config-dir=config --config-name=baseline_image_can_ph_diffusion_policy_transformer.yaml hydra.run.dir='data/outputs/${now:%Y.%m.%d}/original_dp/${now:%H.%M.%S}_${name}_${task_name}'

* EVal original DP
python diffusion_policy/eval.py --checkpoint data/outputs/2025.03.26/06.32.29_train_diffusion_transformer_hybrid_square_image/checkpoints/epoch=1100-test_mean_score=0.500.ckpt --output_dir data/output/eval/robomimic/dp_awe/ --device cuda:0

* Modified eval DP
python diffusion_policy/eval.py --checkpoint data/outputs/2025.03.26/06.32.29_train_diffusion_transformer_hybrid_square_image/checkpoints/epoch=1100-test_mean_score=0.500.ckpt --device cuda:5

* Modified eval all DP ckpts
python diffusion_policy/eval_all.py --checkpoint /home/ubuntu/chd/code/awe/data/outputs/2025.03.30/10.38.40_train_diffusion_transformer_hybrid_can_image/checkpoints --device cuda:5


* ACT

* Visualize waypoints
python example/act_waypoint.py --dataset=data/act/sim_transfer_cube_scripted --err_threshold=0.01 --plot_3d --end_idx=0 

* Save waypoints
python example/act_waypoint.py --dataset=data/act/sim_transfer_cube_scripted --err_threshold=0.01 --save_waypoints 

In [3]:
import os

path = "/home/ubuntu/chd/code/awe/data/outputs/2025.03.26/06.32.29_train_diffusion_transformer_hybrid_square_image/checkpoints/"
print(os.path.dirname(path))


/home/ubuntu/chd/code/awe/data/outputs/2025.03.26/06.32.29_train_diffusion_transformer_hybrid_square_image/checkpoints


In [ ]:
mkdir video
python example/robomimic_waypoint_replay.py --dataset=robomimic/datasets/can/ph/low_dim.hdf5 \
    --record_video --video_path video/can_waypoint.mp4 --task=can \
    --plot_3d --auto_waypoint --err_threshold=0.005